# Create a retreat mask from basins and Calfin

In [ ]:
import geopandas as gp
gp.options.io_engine = "pyogrio"
import pandas as pd
from tqdm.auto import tqdm
import xarray as xr
from geocube.api.core import make_geocube
import numpy as np
import rioxarray as rxr
from joblib import Parallel, delayed
from pathlib import Path
from typing import Union

## Set horizontal grid resolution

In [ ]:
resolution = 450

## Set path to data directory

In [ ]:
data_dir = Path("/Users/andy/Google Drive/My Drive/")

## Read IMBIE and Calfin using geopandas

In [ ]:
imbie = gp.read_file(data_dir / "data/gris-outline/gris-outline-imbie-1980.shp").to_crs("EPSG:3413")
calfin = gp.read_file(data_dir / "data/Calfin/level-1_shapefiles-greenland-termini-closed/termini_1972-2019_Greenland_closed_v1.0.shp")

In [ ]:
geom_valid = calfin.geometry.make_valid()
calfin.geometry = geom_valid

In [ ]:
calfin_dissolved = calfin.dissolve()

In [ ]:
calfin_ds = calfin.set_index("QDate").groupby(by=pd.Grouper(freq='ME'))
nt = len(calfin_ds)

In [ ]:
imbie_union = imbie.union(calfin_dissolved)

In [ ]:
x_min = -653000
x_max = 879700
y_min = -632750
y_max = -3384350
bbox = [x_min, y_min, x_max, y_max]
geom = {
    "type": "Polygon",
    "crs": {"properties": {"name": "EPSG:3413"}},
    "bbox": bbox,
    "coordinates": [[
        (x_min, y_min), 
        (x_max, y_min), 
        (x_max, y_max), 
        (x_min, y_max),
        (x_min, y_min)  # Close the loop by repeating the first point
    ]]
}

In [ ]:
def create_ds(ds1, ds2, date, resolution: float = 450, crs: str = "EPSG:3413", like: Union[None, xr.Dataset] = None):
    import cf_xarray
    if len(ds1) > 0:
        diff = ds2.difference(ds1.dissolve().buffer(5))
        n = len(diff)
        diff_df = {"land_ice_area_fraction_retreat": np.ones(n)}
        diff_gp = gp.GeoDataFrame(data=diff_df, geometry=diff, crs=crs)
        ds = make_geocube(vector_data=diff_gp, geom=geom, resolution=(resolution, resolution))
        ds.land_ice_area_fraction_retreat["units"] = "1"
        ds = ds.expand_dims(time=[date])
        fn = Path(f"frontretreat_g{resolution}m_{date.year}-{date.month}.nc", encoding={"zlib": True, "complevel": 2})
        ds.to_netcdf(fn)
        del ds
        return fn

In [ ]:
n_jobs = 8
result = Parallel(n_jobs=n_jobs)(
    delayed(create_ds)(ds,
        imbie_union, date, resolution=resolution)
    for date, ds in tqdm(calfin_ds))

## Merge files and add time bounds

This could probably be done with *xarray* but setting the time axis and time_bounds correctly appears hard.

In [ ]:
fn = Path(f"pism_g{resolution}m_frontretreat_calfin_1972_2019.nc")
!cdo -f nc4 -z zip_2 settbounds,1month -setmisstoc,0 -mergetime frontretreat_g450m_* $fn